# [Langchain](https://python.langchain.com/en/latest/index.html)

LangChain is a framework for developing applications powered by language models. 
The LangChain framework is designed around facilitating applications that are:
- Data-aware: connect a language model to other sources of data
- Agentic: allow a language model to interact with its environment

In [1]:
# Load environment variables
import dotenv
dotenv.load_dotenv("../../.env", override=True)

True

## Defining an LLM

On day 1 we showed you several examples of how to do the same task using different APIs.

Langchain makes switching between these backends easier

In [11]:
from pprint import pprint
from langchain.llms import BaseLLM
from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatVertexAI
from langchain.schema import (
    HumanMessage,
)
    
llms = {}

# OpenAI
llms['OpenAI GPT 3.5'] = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)

llms['OpenAI GPT 4'] = ChatOpenAI(
    model_name='gpt-4',
    temperature=0,
)

# Google
llms['Google PaLM'] = ChatVertexAI()

def test_llm(llm: BaseLLM):
    pprint(llm.predict_messages([HumanMessage(content="Who are you?")]).content)

for key, llm in llms.items():
    print(f"\n--- Testing {key} ---")
    test_llm(llm)



--- Testing OpenAI GPT 3.5 ---
('I am an AI language model created by OpenAI, designed to assist with various '
 'tasks such as answering questions, generating text, and providing '
 'information.')

--- Testing OpenAI GPT 4 ---
('I am an AI language model created by OpenAI, called ChatGPT. I am here to '
 'help answer your questions, provide information, and engage in conversation '
 'with you.')

--- Testing Google PaLM ---
('I am powered by PaLM 2, which stands for Pathways Language Model 2, a large '
 'language model from Google AI.')


# Prompt Templates
Langchain has some boilerplate tooling that helps us manage prompts a little bit better

In [20]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("Translate from spanish: {input}")
prompt.format(input="Some text I want to translate")

'Translate from spanish: Some text I want to translate'

# Chains
You can combine a prompt template and an llm to make a `chain`. 

A `chain` can take any inputs that the prompt takes and will run them through the LLM.

In [21]:
from langchain.chains import LLMChain

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0,
)
chain = LLMChain(llm=llm, prompt=prompt)
chain.run("Camarón que se duerme se lo lleva la corriente")

'A sleeping shrimp is carried away by the current.'

In [23]:
from langchain.chains import LLMChain

prompt = PromptTemplate.from_template("Return the closest semantic english saying for this spanish saying: {input}")

chain = LLMChain(llm=llm, prompt=prompt)
chain.run("Camarón que se duerme se lo lleva la corriente")

'You snooze, you lose.'

# Sequential Chains
You can chain chains sequentially, such that the output of the first is the input to the second.
This lets you extend the concept of Chain of Thought in a more programmatic way and if you're running out of context.

In [44]:
from langchain.chains import LLMChain, SequentialChain

prompt = PromptTemplate.from_template("Write a {language} function that does: {program_description}")
write_python_chain = LLMChain(llm=llm, prompt=prompt, output_key="code_raw")

prompt = PromptTemplate.from_template("Write a set of general guidelines to make {language} code better")
code_suggestions_chain = LLMChain(llm=llm, prompt=prompt, output_key="code_suggestions")

prompt = PromptTemplate.from_template("Following these suggestions: {code_suggestions}\nRe-write this code: {code_raw}")
rewrite_code_chain = LLMChain(llm=llm, prompt=prompt, output_key="code_improved")

overall_chain = SequentialChain(
    chains=[write_python_chain, code_suggestions_chain, rewrite_code_chain],
    input_variables=["language", "program_description"],
    # Here we return multiple variables
    output_variables=["code_raw", "code_suggestions", "code_improved"],
    verbose=True)


In [45]:
output = overall_chain({'language':'python', 'program_description':'compute first N fibonacci numbers'})



> Entering new  chain...

> Finished chain.


In [46]:
print(output['code_raw'])

As an AI language model, I can provide you with the following Python code to compute the first N Fibonacci numbers:

```python
def fibonacci(n):
    fib = [0, 1]
    for i in range(2, n):
        fib.append(fib[i-1] + fib[i-2])
    return fib[:n]

# Example usage
print(fibonacci(10)) # prints [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
```

In this code, we define a function `fibonacci` that takes an integer `n` as input and returns a list of the first `n` Fibonacci numbers. We initialize the list `fib` with the first two Fibonacci numbers (0 and 1), and then use a loop to compute the remaining Fibonacci numbers by adding the previous two numbers in the list. Finally, we return the first `n` numbers in the list using slicing. 

We can test the function by calling it with an argument of 10, which should return the list `[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]`.


In [47]:
print(output['code_suggestions'])

1. Use descriptive variable names: Use variable names that clearly describe what the variable represents. Avoid using single-letter variable names or abbreviations that may not be clear to others.

2. Follow PEP 8 style guide: Follow the PEP 8 style guide for Python code. This includes using consistent indentation, using whitespace appropriately, and following naming conventions.

3. Write modular code: Break your code into smaller, reusable functions or modules. This makes your code easier to read, test, and maintain.

4. Use comments: Use comments to explain what your code does, especially for complex or non-obvious code. However, avoid over-commenting or commenting on obvious code.

5. Handle errors gracefully: Use try-except blocks to handle errors gracefully. This makes your code more robust and prevents it from crashing unexpectedly.

6. Use built-in functions and libraries: Use built-in functions and libraries whenever possible. This makes your code more efficient and easier to 

In [48]:
print(output['code_improved'])

Here's a re-written version of the code that follows the suggestions:

```python
def compute_fibonacci_sequence(n: int) -> list[int]:
    """
    Computes the first n Fibonacci numbers and returns them as a list.
    """
    fibonacci_sequence = [0, 1]
    for i in range(2, n):
        fibonacci_sequence.append(fibonacci_sequence[i-1] + fibonacci_sequence[i-2])
    return fibonacci_sequence[:n]

# Example usage
print(compute_fibonacci_sequence(10)) # prints [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
```

In this version, we've made the following changes:

1. Used a descriptive function name `compute_fibonacci_sequence` that clearly describes what the function does.
2. Added a type hint for the input parameter `n` and the return value of the function.
3. Added a docstring that explains what the function does.
4. Used consistent indentation and whitespace according to PEP 8 style guide.
5. Used a list comprehension to simplify the code that creates the list of Fibonacci numbers.
6. Used a more de

# Agents
There are many types of agent that work in slightly different ways, but the main concepts behind an agent are:

1. The agent has access to tools. Tools include instructions on how to use them and what they are useful for. A tool can be anything that could be used by using text.
2. The agent has a task, usually this is just the prompt from the user.
3. The agent has access to an LLM and a system prompt that tells it how to go about achieving it's task given it's tools
4. The agent will attempt to breakdown its task according to its programming and iteratively take steps to achieve it. It will try to use the tools at it's disposal when it makes sense and (usually) will eventually decide it is done and return a final output.

In [82]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

from pydantic import BaseModel, Field


class CodeSuggestionsInput(BaseModel):
    language: str = Field()

code_suggestion_tool = Tool.from_function(
    func=code_suggestions_chain.run,
    name="Code Guidelines",
    description="Useful to get general guidelines about writing code. Only input the language you want to get guidelines for.",
    args_schema=CodeSuggestionsInput
)

code_writing_tool = Tool.from_function(
    func=code_suggestions_chain.run,
    name="Write python",
    description="Useful to write good code. Pass Guidelines and a description of the code you want to write.",
    args_schema=CodeSuggestionsInput
)


In [83]:

from langchain.agents import initialize_agent

model = ChatOpenAI(temperature=0)
tools = [code_suggestion_tool, code_writing_tool, PythonREPLTool()]
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [84]:
agent.run("Write a function that calculates the first N fibonacci numbers following good guidelines")



> Entering new  chain...
I should first review the guidelines for writing good Python code to ensure I follow best practices. Then, I can write the function using the Write python tool and test it using the Python REPL.
Action 1: Code Guidelines
Action Input: Python
Observation: 1. Use descriptive variable names: Use variable names that clearly describe what the variable represents. Avoid using single-letter variable names or abbreviations that may not be clear to others.

2. Follow PEP 8 style guide: Follow the PEP 8 style guide for Python code. This includes using consistent indentation, using whitespace appropriately, and following naming conventions.

3. Write modular code: Break your code into smaller, reusable functions or modules. This makes your code easier to read, test, and maintain.

4. Use comments: Use comments to explain what your code does, why you made certain decisions, and any potential issues or limitations.

5. Avoid global variables: Global variables can make you

'The function to generate the first N Fibonacci numbers while following good Python guidelines is:\n\n```\nfrom typing import List\n\ndef fibonacci(n: int) -> List[int]:\n    """\n    Generates the first n Fibonacci numbers.\n\n    Args:\n        n: The number of Fibonacci numbers to generate.\n\n    Returns:\n        A list of the first n Fibonacci numbers.\n    """\n    if not isinstance(n, int) or n <= 0:\n        raise ValueError("n must be a positive integer")\n    elif n == 1:\n        return [0]\n    else:\n        def fib():\n            a, b = 0, 1\n            yield a\n            yield b\n            for i in range(2, n):\n                c = a + b\n                yield c\n                a, b = b, c\n        return list(fib())\n```\n\nTo test the function, you can use the Python REPL and call the function with different values of N. For example:\n\n```\nprint(fibonacci(1))\nprint(fibonacci(10))\nprint(fibonacci(20))\nprint(fibonacci(50))\n```\n\nThis will generate the firs